In [1]:
# Core analysis packages
import numpy as np
import os, sys
import pandas as pd
from scipy import stats
from scipy.special import comb
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats import anova
from patsy import dmatrices
import bff

import networkx as nx

# Plotting packages
import matplotlib.pyplot as plt; plt.rcdefaults()
import seaborn as sns 
sns.set(style="ticks", color_codes=True)
sns.set_style("white")
sns.set_style({'xtick.bottom': True, 'ytick.left': True})
colorref = ["gray", "royalblue", "crimson", "goldenrod", "mediumorchid", "seagreen"]

# iPython magic commands
%matplotlib notebook
%load_ext autoreload
%autoreload 2
%autosave 30

SMALL_SIZE = 12
MEDIUM_SIZE = 12
BIG_SIZE = 14

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIG_SIZE)  # fontsize of the figure title
cust_palette = sns.color_palette("Paired")[6:10]
cust_palette = [cust_palette[i] for i in [1,0,3,2]]

def median_split(S):
    return S > S.median()

Autosaving every 30 seconds


In [2]:
# Load file (from same directory as the notebook)
df = pd.read_excel(os.path.expanduser("network.xlsx"), index_col=0)
df.head()

,CLUSTER,NETWORK,PARTICIPANT,CONDITION,DURATION,GENDER,AGE,AMERICAN,PARTY,TWITTERUSER,...,round7,round8,round9,round10,round11,round12,round13,round14,round15,round16
Response ID,,,,,,,,,,,,,,,,,,,,,
R_2xRSW7gQOO8Aenr,1,1,1,1,2140,1,19,0,3,1,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
R_1g5V0U7NatFob0x,1,1,2,1,2514,0,20,0,4,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
R_3iwBoUcO1237jF1,1,1,3,1,2428,1,19,1,1,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
R_1QoMZGGj8soR7JL,1,1,4,1,2365,0,20,1,1,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
R_3R2sjtnUIGMT7o5,1,1,5,1,2329,0,18,1,2,1,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Extract columns with df.target or df["target"] or df.loc[:,"target"]
age = df.loc[:, "AGE"]
party = df.loc[:, "PARTY"]
twitter = df.loc[:, "TWITTERUSER"]
trust = df.loc[:,"TRUSTSCIENCE"]
duration = df.loc[:,"DURATION"]
network = df.loc[:,"NETWORK"]
participant = df.loc[:,"PARTICIPANT"]

In [4]:
# Extract data with df.loc[:,"datastart":"dataend"]
RdeltaB = df.loc[:, "RU1":"RU16"]
convincing = df.loc[:, "C1":"C16"]
rigorous = df.loc[:, "R1":"R16"]
personal = df.loc[:, "P1":"P16"]
widespread = df.loc[:, "W1":"W16"]
share = df.loc[:, "S1":"S16"]
time = df.loc[:, "T1":"T16"]
selfM = df.loc[:, "selfM1":"selfM16"]
jointM = df.loc[:, "jointM1":"jointM16"]
partnerM = df.loc[:, "partnerM1":"partnerM16"]
selfB = df.loc[:, "selfB1":"selfB16"]
jointB = df.loc[:, "jointB1":"jointB16"]
partnerB = df.loc[:, "partnerB1":"partnerB16"]
pre = df.loc[:, "PRE1":"PRE16"]
post = df.loc[:, "POST1":"POST16"]


key = [[0,1,2,3,0,1,2,3,4,4,4,4,4,4,4,4],
       [1,2,3,0,1,2,3,0,4,4,4,4,4,4,4,4],
       [2,3,0,1,2,3,0,1,4,4,4,4,4,4,4,4],
       [3,0,1,2,3,0,1,2,4,4,4,4,4,4,4,4]]
ticklbl = ["Anec, Pop","Anec, Unpop","Sci, Pop","Sci, Unpop","Base"]

cond = np.array([key[i-1] for i in df["CONDITION"]])
sz = RdeltaB.shape
partnum = np.arange(sz[0])
itemnum = np.arange(sz[-1])

# This code below is equivalent to the list comprehension above:
#temp = []
#for i in df["CONDITION"]:
#    temp += [key[i-1]]

In [7]:
#create a list of names for the column headers
names = ["RdeltaB", "convincing", "rigorous", "personal", "widespread", "share", "time", "selfM", "partnerM", "jointM", "selfB", "partnerB", "jointB","pre","post", "cond", "itemnum", \
         "age", "party", "twitter", "trust", "partnum", "duration", "network", "participant"]

#make all the participantXitem variables the same size with numpy arrays broadcast function
data_packed = np.broadcast_arrays(RdeltaB, convincing, rigorous, personal, widespread, share, time, selfM, partnerM, jointM, selfB, partnerB, jointB, pre, post, cond, itemnum)

#make all the participant variables the same size with a similar function to boradcast
cols = [np.tile(a, (sz[1],1)).T for a in [age, party, twitter, trust, partnum, duration, network, participant]]

#append cols to data_packed
data_packed += cols

#take all this and shape it into long format
data_unpacked = np.vstack([np.reshape(a, (1,-1), order="C") for a in data_packed]).T

#make it a dataframe with the names created above
DATA = pd.DataFrame(data=data_unpacked, columns=names)

nparticipant = DATA["partnum"].max().astype(int)+1
#compute and insert low-moderate-high quantization of belief@pre
lmh_all = []
lmh = [0]*5 + [1]*6 + [2]*5
for p in range(nparticipant):
    idx = np.argsort(DATA.loc[DATA["partnum"]==p, "pre"])
    lmh_sorted = [x for _, x in sorted(zip(idx, lmh))]
    lmh_all += lmh_sorted
DATA.insert(14, "LMH", lmh_all)

#tag rational increases, decreases, baselines
idb = [0]*4 + [1]*4 + [2]*8
idb *= nparticipant
DATA.insert(17, "incdec", idb)

#tag evidence vs baseline
evbase = [0]*8 + [1]*8
evbase *= nparticipant
DATA.insert(1, "evbase", evbase)

#look at the dataframe in long format
DATA.head()

,RdeltaB,evbase,convincing,rigorous,personal,widespread,share,time,selfM,partnerM,...,incdec,itemnum,age,party,twitter,trust,partnum,duration,network,participant
0,-30.0,0,69.0,55.0,87.0,55.0,28.0,28.413,0.0,0.0,...,0,0.0,19.0,3.0,1.0,81.0,0.0,2140.0,1.0,1.0
1,-1.0,0,37.0,60.0,68.0,38.0,33.0,32.326,0.0,0.0,...,0,1.0,19.0,3.0,1.0,81.0,0.0,2140.0,1.0,1.0
2,-9.0,0,70.0,55.0,54.0,72.0,38.0,34.916,0.0,0.0,...,0,2.0,19.0,3.0,1.0,81.0,0.0,2140.0,1.0,1.0
3,7.0,0,7.0,9.0,17.0,27.0,0.0,34.315,0.0,0.0,...,0,3.0,19.0,3.0,1.0,81.0,0.0,2140.0,1.0,1.0
4,-8.0,0,77.0,56.0,97.0,80.0,64.0,31.454,0.0,0.0,...,1,4.0,19.0,3.0,1.0,81.0,0.0,2140.0,1.0,1.0


# R

In [12]:
%load_ext rpy2.ipython
# %R library(lme4)

#import warnings
#warnings.filterwarnings('ignore')

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [11]:
%R library(lmerTest)

array(['lmerTest', 'lme4', 'Matrix', 'tools', 'stats', 'graphics',
       'grDevices', 'utils', 'datasets', 'methods', 'base'], dtype='<U9')

In [10]:
%Rpush DATA

## Network convergence v1: Change in (rank) correlation from pre to post

In [13]:
nbin = 20
clustered = np.array([1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0], dtype=np.int)

## PRE
corrs_pre = list()
for i in range(max(df["NETWORK"])):
    net = np.array(df.loc[df["NETWORK"]==i+1, "PRE1":"PRE8"])
    corrs_pre += [stats.spearmanr(net.T).correlation]
corrs_pre = np.stack(corrs_pre, axis=0)
## POST
corrs_post = list()
for i in range(max(df["NETWORK"])):
    net = np.array(df.loc[df["NETWORK"]==i+1, "POST1":"POST8"])
    corrs_post += [stats.spearmanr(net.T).correlation]
corrs_post = np.stack(corrs_post, axis=0)

## POST-PRE
cdiff = (corrs_post - corrs_pre) / 2
cdiff_clustered = np.mean(cdiff[clustered==1,:,:], axis=0)
cdiff_nonclustered = np.mean(cdiff[clustered==0,:,:], axis=0)

In [14]:
## Hypothesis matrices
H_clustered = np.array(
    [[0,1,1,1,2,3,4,4,5,5],
     [1,0,1,1,2,3,4,4,5,5],
     [1,1,0,2,1,2,3,3,4,4],
     [1,1,2,0,1,2,3,3,4,4],
     [2,2,1,1,0,1,2,2,3,3],
     [3,3,2,2,1,0,1,1,2,2],
     [4,4,3,3,2,1,0,2,1,1],
     [4,4,3,3,2,1,2,0,1,1],
     [5,5,4,4,3,2,1,1,0,1],
     [5,5,4,4,3,2,1,1,1,0]])
H_nonclustered = np.array(
    [[0,2,1,1,2,3,2,2,1,3],
     [2,0,1,1,2,3,2,2,3,1],
     [1,1,0,2,1,2,3,3,2,2],
     [1,1,2,0,1,2,3,3,2,2],
     [2,2,1,1,0,1,2,2,3,3],
     [3,3,2,2,1,0,1,1,2,2],
     [2,2,3,3,2,1,0,2,1,1],
     [2,2,3,3,2,1,2,0,1,1],
     [1,3,2,2,3,2,1,1,0,2],
     [3,1,2,2,3,2,1,1,2,0]])

In [15]:
iu = np.triu_indices(cdiff_clustered.shape[0], 1)
def shuffle_bootstrap(conv, H, NBOOT=10000):
    map_dict = {i+1: q for i,q in enumerate(np.quantile(conv[iu], np.linspace(0,1,np.max(H))))}
    map_dict[0] = np.nan
    mapper = np.vectorize(lambda x: map_dict[x])
    sim = mapper(H)
    
    r_boot = np.zeros(NBOOT)
    for i in range(NBOOT):
        perm = np.random.permutation(np.arange(10))
        shuffled = sim[perm[:,None], perm]
        r_boot[i] = stats.pearsonr(shuffled[iu], conv[iu])[0]

    r_obs = stats.pearsonr(sim[iu], conv[iu])[0]
    p = np.sum(np.less(r_boot, r_obs)) / NBOOT
    return p, r_obs, r_boot

clustmatch       = shuffle_bootstrap(cdiff_clustered, H_clustered)
clustmismatch    = shuffle_bootstrap(cdiff_clustered, H_nonclustered)
nonclustmatch    = shuffle_bootstrap(cdiff_nonclustered, H_nonclustered)
nonclustmismatch = shuffle_bootstrap(cdiff_nonclustered, H_clustered)

In [16]:
# Collect network statistics into matrices to match corrs_*, broadcasting as necessary
_, network_id, network_type = np.broadcast_arrays(corrs_pre, 
                                                  np.arange(14)[:, None, None], clustered[:, None, None])
H = np.stack([H_clustered if i==1 else H_nonclustered for i in clustered], axis=0)
# Build dataframe out of these network-level statistics
labels = ["corr_pre", "corr_post", "corr_change", "deg_sep", "net_type", "net_id"]
cols = np.vstack([np.hstack([c[iu] for c in corrs_pre]),
                  np.hstack([c[iu] for c in corrs_post]),
                  np.hstack([c[iu] for c in cdiff]),
                  np.hstack([h[iu] for h in H]),
                  np.hstack([n[iu] for n in network_type]),
                  np.hstack([n[iu] for n in network_id])]).T # transpose to "column form"
NETDATA = pd.DataFrame(cols, columns=labels)
NETDATA.head(10)

,corr_pre,corr_post,corr_change,deg_sep,net_type,net_id
0,0.131739,-0.047619,-0.089679,1.0,1.0,0.0
1,0.403614,0.287430,-0.058092,1.0,1.0,0.0
2,0.550908,0.239525,-0.155691,1.0,1.0,0.0
3,0.895857,0.658694,-0.118581,2.0,1.0,0.0
4,0.047905,0.243975,0.098035,3.0,1.0,0.0
5,-0.186747,-0.167668,0.009540,4.0,1.0,0.0
6,-0.060241,0.550908,0.305575,4.0,1.0,0.0
7,0.503003,0.547619,0.022308,5.0,1.0,0.0
8,-0.006024,0.000000,0.003012,5.0,1.0,0.0
9,0.658694,0.299407,-0.179644,1.0,1.0,0.0


In [17]:
%Rpush NETDATA

In [18]:
%%R

M <- lmer(corr_change ~ -1 + as.factor(deg_sep) + (1 | net_id), data = NETDATA)
print(summary(M))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: corr_change ~ -1 + as.factor(deg_sep) + (1 | net_id)
   Data: NETDATA

REML criterion at convergence: -138.1

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.4328 -0.6638  0.0013  0.6567  3.1235 

Random effects:
 Groups   Name        Variance Std.Dev.
 net_id   (Intercept) 0.001086 0.03296 
 Residual             0.044570 0.21112 
Number of obs: 630, groups:  net_id, 14

Fixed effects:
                      Estimate Std. Error         df t value Pr(>|t|)   
as.factor(deg_sep)1   0.057805   0.017024  48.543915   3.395  0.00137 **
as.factor(deg_sep)2   0.026496   0.017156  47.197884   1.544  0.12917   
as.factor(deg_sep)3   0.009718   0.020780 101.325113   0.468  0.64103   
as.factor(deg_sep)4  -0.034242   0.030235 230.876855  -1.133  0.25859   
as.factor(deg_sep)5   0.032807   0.041353 447.013587   0.793  0.42799   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ 

In [19]:
# Collect network statistics into matrices to match corrs_*, broadcasting as necessary
_, network_id, network_type = np.broadcast_arrays(corrs_pre, 
                                                  np.arange(14)[:, None, None], clustered[:, None, None])
H = np.stack([H_clustered if i==1 else H_nonclustered for i in clustered], axis=0)

sz = np.hstack([c[iu] for c in corrs_pre]).shape
corrs_all = np.hstack([c[iu] for c in corrs_pre]+[c[iu] for c in corrs_post])
prepost   = np.hstack([np.zeros(sz), np.ones(sz)])
H_twice   = np.tile(np.hstack([h[iu] for h in H]), 2)
nt_twice  = np.tile(np.hstack([n[iu] for n in network_type]), 2)
nid_twice = np.tile(np.hstack([n[iu] for n in network_id]), 2)

# Build dataframe out of these network-level statistics
labels = ["corr", "pre_post", "deg_sep", "net_type", "net_id"]
cols = np.vstack([corrs_all, prepost, H_twice, nt_twice, nid_twice]).T

PPDATA = pd.DataFrame(cols, columns=labels)
PPDATA.head()

,corr,pre_post,deg_sep,net_type,net_id
0,0.131739,0.0,1.0,1.0,0.0
1,0.403614,0.0,1.0,1.0,0.0
2,0.550908,0.0,1.0,1.0,0.0
3,0.895857,0.0,2.0,1.0,0.0
4,0.047905,0.0,3.0,1.0,0.0


In [21]:
%R library(statnet)

array(['statnet', 'tsna', 'sna', 'statnet.common', 'ergm.count', 'tergm',
       'networkDynamic', 'ergm', 'network', 'lmerTest', 'lme4', 'Matrix',
       'tools', 'stats', 'graphics', 'grDevices', 'utils', 'datasets',
       'methods', 'base'], dtype='<U14')

In [22]:
%Rpush PPDATA

In [23]:
%%R

M <- lmer(corr ~ -1 + pre_post + (1 | net_id), data = PPDATA)
print(summary(M))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: corr ~ -1 + pre_post + (1 | net_id)
   Data: PPDATA

REML criterion at convergence: 1122.5

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-2.62171 -0.70122  0.02675  0.72132  2.69479 

Random effects:
 Groups   Name        Variance Std.Dev.
 net_id   (Intercept) 0.001988 0.04458 
 Residual             0.140818 0.37526 
Number of obs: 1260, groups:  net_id, 14

Fixed effects:
          Estimate Std. Error        df t value Pr(>|t|)    
pre_post   0.06180    0.01762 139.86586   3.508 0.000607 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [24]:
%Rpush NETDATA

In [25]:
%%R

M <- lmer(corr_change ~ -1 + net_type + (1 | net_id), data = NETDATA)
print(summary(M))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: corr_change ~ -1 + net_type + (1 | net_id)
   Data: NETDATA

REML criterion at convergence: -147.9

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.4881 -0.6171  0.0217  0.6939  3.2573 

Random effects:
 Groups   Name        Variance Std.Dev.
 net_id   (Intercept) 0.001468 0.03831 
 Residual             0.045012 0.21216 
Number of obs: 630, groups:  net_id, 14

Fixed effects:
         Estimate Std. Error       df t value Pr(>|t|)
net_type  0.02860    0.01878 13.00000   1.523    0.152


In [26]:
%%R

M <- lmer(RdeltaB ~ -1 + jointM + jointB + (1 | partnum) + (1 | network), data = DATA)
print(summary(M))


R[write to console]: boundary (singular) fit: see ?isSingular



Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: RdeltaB ~ -1 + jointM + jointB + (1 | partnum) + (1 | network)
   Data: DATA

REML criterion at convergence: 9925

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.8093 -0.4243  0.0130  0.4817  4.1306 

Random effects:
 Groups   Name        Variance Std.Dev.
 partnum  (Intercept)  15.37    3.921  
 network  (Intercept)   0.00    0.000  
 Residual             400.26   20.007  
Number of obs: 1120, groups:  partnum, 140; network, 14

Fixed effects:
        Estimate Std. Error        df t value Pr(>|t|)    
jointM    0.9201     0.4918  950.5268   1.871   0.0617 .  
jointB    1.7378     0.3516 1099.1868   4.942 8.93e-07 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
       jointM
jointB -0.287
convergence code: 0
boundary (singular) fit: see ?isSingular

